In [1]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModel, Trainer, TrainingArguments, BertForMaskedLM, AutoModelForCausalLM
import torch
from torch.utils.data import DataLoader, TensorDataset
from datasets import Dataset
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tqdm.notebook import tqdm, trange
import numpy as np
from scipy.spatial.distance import cosine
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GenerationConfig
import os
import json
from langchain_core.embeddings import Embeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from torch import nn
from torch import optim
from sklearn.model_selection import train_test_split

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
QDRANT_URL = "http://213.171.5.51:6333"
QDRANT_COLLECTION_NAME = "FULL_UPLOAD"
SIZE=1024

## Обучение модели для определения ответа

Есть идея, что нам необязательно использовать ЛЛМ для финального вердикта, достаточно обучить модель для соответствия запроса и его контекста какому-то ответу. Этот подход может быть очень перспективным, так как предлагает много способов для обучения, а так же является боле быстрым и более надежным, чем стандартные ЛЛМ. Также, можно учитывать BERT Score. К сожалению, попробовать внедрить данный подход во время хакатона не удалось, но проверить эту идею было бы интересно в рамках масштабирования проекта

In [4]:
learning_rate = 0.001
num_epochs = 100

In [5]:
model = AutoModelForCausalLM.from_pretrained("models2").to(device)
tokenizer = AutoTokenizer.from_pretrained("models2")

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


In [6]:
class CustomEmbedding(Embeddings):
    def __init__(self, directory: str = "models2"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = BertForMaskedLM.from_pretrained(directory)
        self.tokenizer = AutoTokenizer.from_pretrained(directory)
        self.model.to(self.device)

    def embed_query(self, text: str) -> list[float]:
        return extract_features(text, self.model, self.tokenizer).tolist()

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        return list(map(lambda text: self.embed_query(text), texts))


def extract_features(text, model, tokenizer):
    model.eval()

    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        tokenizer,
        chunk_size=512,
        chunk_overlap=64
    )

    chunks = text_splitter.split_text(text)

    features = []

    for chunk in chunks:
        inputs = tokenizer(chunk, return_tensors="pt", truncation=True, padding="max_length", max_length=512).to(device)
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)
        last_hidden_state = outputs.hidden_states[-1]

        attention_mask = inputs['attention_mask']
        mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size())
        sum_embeddings = torch.sum(last_hidden_state * mask_expanded, 1)
        mean_embeddings = sum_embeddings / torch.clamp(mask_expanded.sum(1), min=1e-9)

        features.append(mean_embeddings)

    return torch.stack(features).mean(dim=0).squeeze(0).cpu()


def get_context(question: str, k: int = 20) -> list[str]:
    client = QdrantClient(url=QDRANT_URL)

    if not client.collection_exists(QDRANT_COLLECTION_NAME):
        client.create_collection(QDRANT_COLLECTION_NAME, vectors_config=VectorParams(size=SIZE, distance=Distance.COSINE))

    
    embedding = CustomEmbedding()

    qdrant = QdrantVectorStore(
        client=client,
        collection_name=QDRANT_COLLECTION_NAME,
        embedding=embedding,
    )

    return [i.page_content for i in qdrant.similarity_search(question, k=k)]

Наша архитектура нейронки:

In [7]:
class VectorMatchingModel(torch.nn.Module):
    def __init__(self):
        super(VectorMatchingModel, self).__init__()
        self.fc1 = nn.Linear(1024 * 21, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 1024)

        self.act = nn.LeakyReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        x = self.act(x)
        x = self.fc3(x)
        return x

In [8]:
vmmodel = VectorMatchingModel().to(device)

In [9]:
criterion = nn.MSELoss()
optimizer = optim.Adam(vmmodel.parameters(), lr=learning_rate)

In [10]:
dataset = []

In [11]:
questions = [
    "Какие существуют типы источников выбросов? Привести примеры каждого источника.",
    "Расписать состав тома инвентаризации",
    "Для каких предприятий разрабатывается инвентаризация?",
    "Как присваиваются номера ИВ?",
    "Какие бывают методы определения качественных и количественных характеристик ИЗАВ?",
    "Какие источники мы можем рассчитать, а для каких необходимо производить измерения? Определение качественного и количественного состава выбросов из выявленных ИЗАВ осуществляется инструментальными и расчетными методами.",
    "Как задается высота организованных и неорганизованных источников? Какие высоты будут у стоянка спецтехники, перегрузка сырья на открытом складе, труба от котельной, дверь цеха.",
    "Напишите приблизительный перечень источников выбросов загрязняющих веществ от авторемонтной мастерской",
    "Напишите приблизительный перечень источников выбросов загрязняющих веществ от угольной котельной",
    "При каких параметрах рассеивания загрязняющих веществ необходим учет фонового загрязнения атмосферного воздуха?",
    "Что значит «многоступенчатая очистка ГВС»?",
    "Каков порядок нумерации источников выбросов загрязняющих веществ (ИЗАВ) при проведении первой инвентаризации на объекте?",
    "Какие типы ИЗАВ существуют и что к ним относится?",
    "Какие сведения определяются и документируются при проведении инвентаризации?",
    "Какие требования существуют к графическим материалам в составе отчета об инвентаризации?",
    "Какие есть особенности у ИЗАВ, оснащенных установками очистки газа, и какие есть особенности в отражении таких ИЗАВ в отчете об инвентаризации?",
    "В каких случаях применяются те или иные методы определения количественных характеристик выбросов загрязняющих веществ?",
    "Что входит в обязательный состав отчета об инвентаризации источников выбросов?",
    "Что является источником выделения загрязняющих веществ?", 
    "Для каких объектов требуется разработка и установление нормативов допустимых выбросов загрязняющих веществ в атмосферный воздух?",
    "В чем заключается особенность разработки и установления нормативов допустимых выбросов загрязняющих веществ в атмосферный воздух для строящихся объектов?",
    "Что такое фоновый уровень загрязнения атмосферного воздуха и как его используют при разработке проекта НДВ (ПДВ)?",
    "Какие существуют требования к оформлению графических материалов в составе проекта НДВ (ПДВ)?",
    "Каким образом определяется соответствие влияния предприятия гигиеническим нормативам?",
    "На какой срок разрабатываются и устанавливаются нормативы допустимых выбросов загрязняющих веществ в атмосферный воздух?",
    "На основании какого документа (не НПА) разрабатываются и устанавливаются нормативы допустимых выбросов загрязняющих веществ в атмосферный воздух?",
]

In [12]:
answers = [
    "Организованные (дымовые трубы, выходы вентиляции), неорганизованные (открытые склады, двери, окна), передвижные (маршруты следования автотранспорта)",
    """ВВЕДЕНИЕ
    1.СВЕДЕНИЯ О ХОЗЯЙСТВУЮЩЕМ СУБЪЕКТЕ, ЕГО ОТДЕЛЬНЫХ ТЕРРИТОРИЯХ И ВИДАХ ДЕЯТЕЛЬНОСТИ, ВКЛЮЧАЮЩИЕСЯ В ОТЧЕТ О РЕЗУЛЬТАТАХ ИНВЕНТАРИЗАЦИИ ВЫБРОСОВ
    1.1 Характеристика основных технологических процессов, оборудования и иных источников выбросов загрязняющих веществ
    1.2. Характеристика установок очистки газа и оценка их эффективности
    2. ОПИСАНИЕ ПРОВЕДЁННЫХ РАБОТ ПО ИНВЕНТАРИЗАЦИИ ВЫБРОСОВ С УКАЗАНИЕМ НОРМАТИВНО – МЕТОДИЧЕСКИХ ДОКУМЕНТОВ И ПЕРЕЧНЯ ИСПОЛЬЗОВАННЫХ МЕТОДИК ВЫПОЛНЕНИЯ ИЗМЕРЕНИЙ ЗАГРЯЗНЯЮЩИХ ВЕЩЕСТВ И РАСЧЁТНОГО ОПРЕДЕЛЕНИЯ ВЫБРОСОВ ЗАГРЯЗНЯЮЩИХ ВЕЩЕСТВ
    3. КОРРЕКТИРОВКА ДАННЫХ ИНВЕНТАРИЗАЦИИ ВЫБРОСОВ
    4. ХАРАКТЕРИСТИКИ ИЗАВ, ПОКАЗАТЕЛИ РАБОТЫ ГОУ, СУММАРНЫЕ ВЫБРОСЫ ПО ОНВ
    5. РЕЗУЛЬТАТЫ ОПРЕДЕЛЕНИЯ ВЫБРОСОВ ЗВ РАСЧЕТНЫМИ МЕТОДАМИ
    6. ДОКУМЕНТИРОВАНИЕ ХАРАКТЕРИСТИК НЕСТАЦИОНАРНОСТИ ВЫБРОСОВ. 
    7. СПИСОК ИСПОЛЬЗОВАННЫХ ИСТОЧНИКОВ
    ПРИЛОЖЕНИЯ
    Приложение 1. Справочные материалы
    Приложение 2. Графические материалы
    Приложение 3. Результаты определения выбросов ЗВ расчетными методами""",
    "Инвентаризация стационарных источников и выбросов загрязняющих веществ в атмосферный воздух проводится юридическими лицами и индивидуальными предпринимателями, осуществляющими хозяйственную или иную деятельность на объектах, оказывающих негативное воздействие на окружающую среду, с использованием стационарных источников выбросов загрязняющих веществ в атмосферный воздух.",
    "Для каждого ИЗАВ начиная с 01.",
    "Инструментальные (лабораторные) и расчетные (на основании методических указаний)",
    """Для определения показателей выбросов организованных ИЗАВ используются преимущественно инструментальные методы. Случаи, при которых допускается использование расчетных методов для определения качественного и количественного состава выбросов из выявленных ИЗАВ, указаны в пунктах 26-28 настоящего Порядка. В случае использования расчетных методов в отчет о результатах инвентаризации выбросов, предусмотренный главой V настоящего Порядка, включается обоснование выбора и применения использованных методов.
    Для определения показателей выбросов от передвижных ИЗАВ используются преимущественно расчетные методы.
    Использование расчетных методов для определения показателей выбросов организованных ИЗАВ допускается в следующих случаях:
    отсутствие аттестованных методик (методов) измерения загрязняющего вещества;
    отсутствие практической возможности забора проб для определения инструментальными методами в соответствии с национальными стандартами Российской Федерации, утверждаемыми Росстандартом в соответствии с требованиями законодательства Российской Федерации об обеспечении единства измерений;
    отсутствие практической возможности проведения инструментальных измерений выбросов (например, высокая температура газовоздушной смеси, высокая скорость потока отходящих газов, сверхнизкое или сверхвысокое давление внутри газохода, отсутствие доступа к ИЗАВ).
    27. Использование расчетных методов для определения показателей выбросов допускается также для неорганизованных и (или) линейных стационарных ИЗАВ.
    28. Расчетные методы с использованием результатов измерений отдельных характеристик источника выбросов, расхода сырья или топлива, а также загруженности, продолжительности работы оборудования применяются для инвентаризации выбросов:
    от неорганизованных ИЗАВ;
    от топливосжигающих установок мощностью не более 50 МВт;
    при выполнении работ по нанесению металлопокрытий гальваническим способом, по механической обработке материалов, сварочных и окрасочных работ; взрывных работ, погрузочно-разгрузочных работ;
    от инфраструктуры транспортных объектов, дизельных установок, бензоэлектростанций, бензопил и подобного оборудования;
    от источников открытого хранения топлива, сырья, веществ, материалов, отходов, открытых поверхностей испарения;
    от карьеров добычи полезных ископаемых и открытых участков обработки полезных ископаемых;
    от оборудования и технологических процессов, расположенных на открытом воздухе или в производственных помещениях, не оборудованных вентиляционными установками;
    для получения данных о показателях выбросов проектируемых, строящихся и реконструируемых объектов ОНВ.
    29. Для определения показателей выбросов от стационарных ИЗАВ расчетным методом применяются методики расчета выбросов ЗВ, включенные в перечень методик расчета выбросов ЗВ, который формируется и ведется Министерством природных ресурсов и экологии Российской Федерации (далее - методики расчета выбросов).
    Методика расчета выбросов используется в полном соответствии с областью ее применения для указанных в ней технологических процессов, сооружений, устройств, оборудования, а также сырья, материалов, топлива.
    Если перечень методик расчета выбросов, указанный в абзаце первом настоящего пункта, не содержит соответствующей методики расчета выбросов, для определения показателей выбросов расчетным методом используются расчеты на основе материально-сырьевого баланса технологического процесса, физико-химических закономерностей процессов образования выбросов или показателей удельных величин выбросов от однотипного оборудования.
    При проведении расчетов на основе материально-сырьевого баланса технологического процесса, физико-химических закономерностей процессов образования выбросов или показателей удельных величин выбросов от однотипного оборудования, такие расчеты сопровождаются сведениями о месте проведения исследований, исследуемом технологическом оборудовании, режимах работы оборудования во время измерений, программой измерений с описанием методики (методов) измерений, описанием процедуры измерений или подсчета расходуемого сырья и произведенной продукции, а также перечнем используемых источников информации, из которых берутся показатели и исходные данные для расчетов (названия используемых справочников и (или) стандартов).""",
    "Высота организованных ИЗАВ задается в соответствии с реальной высотой устройства (если высота дымовой труба 10 метров, то и в проекте она будет 10 метров). Для неорганизованных ИЗАВ минимальная высота – 2 метра, при этом для транспорта она составляет 5 метров. В случае дверей, ворот, окон и др. – высота принимается по верхней высоты отверстия (если ворота ангара 10 метров, то высота источника – 10 метров).",
    "Источник выбросов – ворота мастерской. Источники выделения – ДВС автотранспорта, заточные станки, зарядка аккумуляторов, сварочные работы, перелив моторного масла, перелив топлива",
    "Источник выбросов – Двери котельной, дымовая труба. Источники выделения – котел на угле, пересыпка угля на склад, хранение угля на складе, пересыпка угля в котел",
    "При достижении на границе предприятия концентрации загрязняющего вещества 0,1 ПДК.",
    "Последовательное прохождение загрязненного воздуха через несколько ГОУ.",
    """При присвоении стационарным ИЗАВ порядковых номеров используется единая, последовательная (сквозная) нумерация:
    отдельных территорий объекта ОНВ - в рамках территории объекта ОНВ в целом;
    цехов - в рамках отдельных территорий объекта ОНВ;
    участков - в рамках территорий цехов;
    стационарных ИЗАВ - в рамках участков, цехов, отдельных территорий объекта ОНВ или объекта ОНВ в целом (при наличии только одной территории);
    источников выделения, режимов (стадий) работы источников выделения, режимов выбросов - в рамках соответствующих стационарных ИЗАВ.
    Нумерация начинается с N 0001 в возрастающей последовательности.
    Всем организованным источникам выбросов присваивают номера от 0001 до 5999, всем неорганизованным источникам - с 6001.
    Принятая нумерация не может быть изменена при проведении следующей инвентаризации выбросов.
    Для объектов ОНВ, на которых функционирует более 50 стационарных ИЗАВ, допускается рассматривать отдельно каждое структурное подразделение (например, промплощадку, цех, обособленную территорию объекта ОНВ) и проводить независимую нумерацию ИЗАВ для этих структурных подразделений. В этом случае номер ИЗАВ, указываемый на карте-схеме, составляется из номера структурного подразделения объекта ОНВ и номера ИЗАВ внутри структурного подразделения. Так, например, первому ИЗАВ в цехе N 1 присваивается номер 1.0001 для организованных источников и 1.6001 - для неорганизованных.
    Нумерация стационарных ИЗАВ на объекте ОНВ может быть многоразрядной, разряды в номере ИЗАВ отделяются друг от друга точкой и обозначают уровень (подуровень) структурного подразделения. Например, номер первого ИЗАВ, относящегося к восьмому участку третьего цеха, расположенного на второй производственной площадке: 2.03.08.0001. При использовании многоразрядной нумерации схема такой нумерации с пояснениями приводится при документировании результатов инвентаризации выбросов в соответствии с главой V настоящего Порядка.
    На реконструируемых или вновь строящихся объектах ОНВ для стационарных ИЗАВ, которые функционируют только в период строительства или реконструкции и в дальнейшем будут ликвидированы, присваиваются номера, начиная с 5501 для организованных ИЗАВ и начиная с 6501 - для неорганизованных ИЗАВ.
    Передвижные ИЗАВ на объекте ОНВ нумеруются отдельно, при нумерации передвижных ИЗАВ используется индекс "п".
    Нумерация передвижных ИЗАВ начинается с N 0001п и ведется в возрастающей последовательности.""",
    """ИЗАВ, из которых загрязняющие вещества поступают в атмосферный воздух через специальные устройства отвода выбросов, например, установки очистки газа, трубы, вентиляционные шахты, газоходы, воздуховоды, факельные устройства (далее - организованные ИЗАВ);
    ИЗАВ, из которых загрязняющие вещества поступают в атмосферный воздух в составе ненаправленных потоков газа, в том числе места загрузки, выгрузки или хранения сырья, материалов, продукции и иных веществ, негерметичности (неплотности) технологического оборудования, из которых вещества поступают в атмосферный воздух, размещенные на территории объекта ОНВ или в санитарно-защитной зоне отходы, имеющие непосредственный контакт с атмосферным воздухом, например, пылящие поверхности хвостохранилищ, карьеров (далее - неорганизованные ИЗАВ);
    ИЗАВ, из которых загрязняющие вещества поступают в атмосферный воздух из установленного отверстия (далее - точечные ИЗАВ, относящиеся к организованным ИЗАВ);
    ИЗАВ, из которых загрязняющие вещества поступают в атмосферный воздух по установленной линии значительной протяженности, например, аэрационные фонари цехов (далее - линейные ИЗАВ);
    ИЗАВ, из которых загрязняющие вещества поступают в атмосферный воздух с установленной ограниченной поверхности или площади (далее - площадные ИЗАВ, относящиеся к неорганизованным ИЗАВ).""",
    """При проведении инвентаризации выбросов:
    обследуется территория объекта ОНВ, анализируется проектная документация объекта ОНВ, виды деятельности (технологии производства), данные предыдущей инвентаризации выбросов, результаты производственного экологического контроля и государственного экологического контроля (надзора) за период действия предыдущей инвентаризации выбросов;
    определяются сооружения, технические устройства, оборудование, технологические или иные процессы, являющиеся источниками образования и выделения загрязняющих веществ (далее также - источники выделения, ИВ), и выявляются все стационарные ИЗАВ, в том числе стационарные источники залповых выбросов, и передвижные ИЗАВ, из которых непосредственно в атмосферный воздух поступает поток газа, содержащий загрязняющие вещества;
    систематизируются сведения о пространственном размещении ИЗАВ на объекте ОНВ;
    изучаются состояние и условия эксплуатации установок очистки газа;
    устанавливаются типы (организованный, неорганизованный), виды (точечный, линейный, площадной), наименование (труба, вентшахта, аэрационный фонарь, дефлектор, свеча и другие) и геометрические характеристики стационарных ИЗАВ (длина, ширина, высота, при наличии устья - вид и размеры устья источника), а также виды передвижных ИЗАВ с указанием их количества и используемого топлива;
    определяются координаты стационарных ИЗАВ, мест эксплуатации, стоянки, обслуживания и ремонта, а также маршрутов перемещения передвижных ИЗАВ по объекту ОНВ;
    определяются показатели выбросов от стационарных и передвижных ИЗАВ, в том числе устанавливается качественный и количественный состав выбросов с учетом всех загрязняющих веществ, которые могут образоваться, выделиться и поступить в атмосферный воздух (список ЗВ и их концентрации), а также для стационарных ИЗАВ определяются показатели отходящих газов (скорость, температура, давление, влажность, плотность, объемный расход и мощность выброса), для стационарных источников залповых выбросов определяются периодичность, продолжительность и условия, при которых возможны залповые выбросы;
    документируются ход и результаты инвентаризации выбросов.""",
    """На карте-схеме с соблюдением принятого масштаба отображаются:
    все сооружения, здания, корпуса, установки на объекте ОНВ, границы его территории, стационарных ИЗАВ с указанием их номеров; маршруты перемещения, места стоянки и обслуживания передвижных ИЗАВ; границы санитарно-защитной зоны, устанавливаемой в соответствии с пунктом 2 статьи 12 Федерального закона от 30.03.1999 N 52-ФЗ "О санитарно-эпидемиологическом благополучии населения", ближайшей жилой застройки, территорий, выделенных в документах градостроительного зонирования, решениях органов местного самоуправления для организации курортных зон, размещения санаториев, домов отдыха, пансионатов, туристских баз, организованного отдыха населения, в том числе пляжей, парков, спортивных баз и их сооружений на открытом воздухе, а также территорий размещения лечебно-профилактических учреждений длительного пребывания больных и центров реабилитации (далее - зоны с особыми условиями использования территорий);
    масштаб (например, 1:500 или 1:1000), направления сторон света и принятая система координат
    Для объектов ОНВ, на которых функционирует более 50 стационарных ИЗАВ, при необходимости также составляется ситуационный план, схематично отображающий положение объекта ОНВ.
    """,
    """На стационарных ИЗАВ, оснащенных установками очистки газа, определение показателей выбросов осуществляется только инструментальными методами в специально оборудованных местах отбора проб, предусмотренных Правилами эксплуатации установок очистки газа и проектной документацией изготовителя установок очистки газа. При этом в качестве установок очистки газа не рассматриваются установки и устройства очистки газа, являющиеся неотъемлемой частью технологического оборудования и (или) не осуществляющие выбросы загрязняющих веществ непосредственно в атмосферный воздух.
    Эффективность работы установок очистки газа, отражающая степень очистки выбросов, рассчитывается в соответствии с Правилами эксплуатации установок очистки газа.
    Фактическая эффективность (степень очистки) установок очистки газа определяется исходя из фактических показателей работы установок очистки газа по результатам измерений, проводимых в соответствии с законодательством Российской Федерации об обеспечении единства измерений.
    Фактический коэффициент обеспеченности очистки газа определяется как отношение времени в часах работы установок очистки газа за год (независимо от степени очистки) ко времени в часах работы технологического оборудования за год, выраженное в процентах.
    """,
    """Для определения показателей выбросов организованных ИЗАВ используются преимущественно инструментальные методы. В случае использования расчетных методов в отчет о результатах инвентаризации выбросов, включается обоснование выбора и применения использованных методов.
    Для определения показателей выбросов от передвижных ИЗАВ используются преимущественно расчетные методы.
    Использование расчетных методов для определения показателей выбросов организованных ИЗАВ допускается в следующих случаях:
    отсутствие аттестованных методик (методов) измерения загрязняющего вещества;
    отсутствие практической возможности забора проб для определения инструментальными методами в соответствии с национальными стандартами Российской Федерации, утверждаемыми Росстандартом в соответствии с требованиями законодательства Российской Федерации об обеспечении единства измерений;
    отсутствие практической возможности проведения инструментальных измерений выбросов (например, высокая температура газовоздушной смеси, высокая скорость потока отходящих газов, сверхнизкое или сверхвысокое давление внутри газохода, отсутствие доступа к ИЗАВ).
    Использование расчетных методов для определения показателей выбросов допускается также для неорганизованных и (или) линейных стационарных ИЗАВ.
    Расчетные методы с использованием результатов измерений отдельных характеристик источника выбросов, расхода сырья или топлива, а также загруженности, продолжительности работы оборудования применяются для инвентаризации выбросов:
    от неорганизованных ИЗАВ;
    от топливосжигающих установок мощностью не более 50 МВт;
    при выполнении работ по нанесению металлопокрытий гальваническим способом, по механической обработке материалов, сварочных и окрасочных работ; взрывных работ, погрузочно-разгрузочных работ;
    от инфраструктуры транспортных объектов, дизельных установок, бензоэлектростанций, бензопил и подобного оборудования;
    от источников открытого хранения топлива, сырья, веществ, материалов, отходов, открытых поверхностей испарения;
    от карьеров добычи полезных ископаемых и открытых участков обработки полезных ископаемых;
    от оборудования и технологических процессов, расположенных на открытом воздухе или в производственных помещениях, не оборудованных вентиляционными установками;
    для получения данных о показателях выбросов проектируемых, строящихся и реконструируемых объектов ОНВ.
    
    Для определения показателей выбросов от стационарных ИЗАВ расчетным методом применяются методики расчета выбросов ЗВ, включенные в перечень методик расчета выбросов ЗВ, который формируется и ведется Министерством природных ресурсов и экологии Российской Федерации.
    Методика расчета выбросов используется в полном соответствии с областью ее применения для указанных в ней технологических процессов, сооружений, устройств, оборудования, а также сырья, материалов, топлива.
    
    Если перечень не содержит соответствующей методики расчета выбросов, для определения показателей выбросов расчетным методом используются расчеты на основе материально-сырьевого баланса технологического процесса, физико-химических закономерностей процессов образования выбросов или показателей удельных величин выбросов от однотипного оборудования.
    При проведении расчетов на основе материально-сырьевого баланса технологического процесса, физико-химических закономерностей процессов образования выбросов или показателей удельных величин выбросов от однотипного оборудования, такие расчеты сопровождаются сведениями о месте проведения исследований, исследуемом технологическом оборудовании, режимах работы оборудования во время измерений, программой измерений с описанием методики (методов) измерений, описанием процедуры измерений или подсчета расходуемого сырья и произведенной продукции, а также перечнем используемых источников информации, из которых берутся показатели и исходные данные для расчетов (названия используемых справочников и (или) стандартов).
    """,
    """Сведения о хозяйствующем субъекте, объекте ОНВ, его отдельных территориях и видах деятельности, включающиеся в отчет о результатах инвентаризации выбросов, содержат:
    полное и сокращенное (при наличии) наименование хозяйствующего субъекта в соответствии с учредительными документами, организационно-правовую форму, адрес, место нахождения, основной государственный регистрационный номер (ОГРН) - для юридического лица; фамилия, имя, отчество (при наличии), место жительства в Российской Федерации, реквизиты основного документа, удостоверяющего личность гражданина Российской Федерации в Российской Федерации, основной государственный регистрационный номер индивидуального предпринимателя (ОГРНИП) - для индивидуального предпринимателя;
    краткое описание видов деятельности на объекте ОНВ, в том числе перечень и краткую характеристику используемого сырья, топливно-энергетических ресурсов, материалов; проектные и фактические виды и объемы производимой продукции, выполнения работ и (или) оказания услуг, в том числе показатели основной деятельности, для которых рассчитаны валовые выбросы; описание основных технологических процессов (при наличии), оборудования и иных источников выбросов ЗВ, а также сведения о количестве, характеристиках и эффективности установок очистки газа и мероприятиях по охране атмосферного воздуха, реализованных в период действия ранее проведенной инвентаризации выбросов;
    сведения о произошедших на объекте ОНВ изменениях, повлекших изменение объемов выбросов загрязняющих веществ в атмосферный воздух, а также причинах таких изменений;
    краткую характеристику прилегающей к объекту ОНВ местности (например, селитебная, промышленная зона, сельхозугодья, болота) с указанием расстояния до ближайшей жилой застройки и зон с особыми условиями использования территорий;
    размеры и границы санитарно-защитной зоны в случае, если ее установление предусмотрено законодательством Российской Федерации в области обеспечения санитарно-эпидемиологического благополучия населения;
    информацию о должностных лицах, ответственных за проведение инвентаризации выбросов.
    """,
    "Сооружения, технические устройства, оборудование, технологические или иные процессы, являющиеся источниками образования и выделения загрязняющих веществ (далее также - источники выделения, ИВ)",
    "При установлении предельно допустимых выбросов учитывается категория объекта, оказывающего негативное воздействие на окружающую среду (объект ОНВ), присвоенная такому объекту ОНВ при его постановке на государственный учет объектов ОНВ, или соответствие планируемых к строительству, реконструкции и вводу в эксплуатацию объектов ОНВ критериям отнесения объектов ОНВ к объектам I, II, III и IV категорий, установленным на основании статьи 4.2. Федерального закона от 10.01.2002 N 7-ФЗ 'Об охране окружающей среды'",
    "Для строящихся, вводимых в эксплуатацию новых и (или) реконструированных объектов ОНВ предельно допустимые выбросы разрабатываются на основе проектной документации.",
    """Данные о фоновом уровне загрязнения атмосферного воздуха (фоновых концентрациях загрязняющих веществ) запрашиваются в организациях федерального органа исполнительной власти в области гидрометеорологии и смежных с ней областях
    Учет фоновой концентрации  при расчете предельно допустимых выбросов осуществляется при выполнении условия (5) за границами земельного участка, на котором расположен объект ОНВ 
    q>0,1ПДК (5)
    """,
    "Карта-схема объекта ОНВ, с указанием масштаба, направлений сторон света и используемой при инвентаризации выбросов системы координат, с нанесенными на данную карту-схему сооружениями, установками и стационарными источниками с указанием номеров, присвоенных при инвентаризации выбросов, границами земельного участка, на котором расположен объект ОНВ; границами санитарно-защитной зоны,",
    """Расчет предельно допустимых выбросов осуществляется исходя из необходимости соблюдения гигиенических и экологических нормативов качества атмосферного воздуха
    Для гигиенических нормативов качества атмосферного воздуха указываются значения максимальных разовых предельно допустимых концентраций загрязняющих веществ в атмосферном воздухе (далее - ПДК м.р.), ориентировочно безопасных уровней воздействия (далее - ОБУВ), среднесуточных и (или) среднегодовых предельно допустимых концентраций загрязняющих веществ в атмосферном воздухе (далее - ПДК с.с., ПДК с.год) при их наличии, а также перечни смесей загрязняющих веществ, обладающих суммацией действия (комбинированным действием).
    На основе результатов расчета общего загрязнения атмосферного воздуха проверяется соблюдение гигиенических и экологических нормативов качества атмосферного воздуха и рассчитываются значения предельно допустимых выбросов.
    """,
    "Результаты расчетов предельно допустимых выбросов для объектов ОНВ оформляются в виде таблицы, в которой для каждого стационарного источника (с указанием подразделения, цеха, участка, к которому относится источник, и номера источника) по каждому загрязняющему веществу (с указанием наименования и кода загрязняющего вещества) на момент разработки предельно допустимых выбросов и на последующие 7 лет отдельно приводятся значения рассчитанных предельно допустимых выбросов, являющихся нормативами допустимых выбросов в соответствии с законодательством в области охраны окружающей среды, а также, при необходимости, временно разрешенных выбросов (г/с и т/год).",
    "Отчет об инвентаризации источников загрязнения атмосферного воздуха"
    
]

In [13]:
len(questions), len(answers)

(26, 26)

In [14]:
dataset = []

for i in range(26):
    dataset.append([questions[i], answers[i]])

In [15]:
epochs= 30

In [16]:
X = []
for element in dataset:
    tmp = [element[0]] + get_context(element[0])
    tmp = torch.stack([extract_features(i, model, tokenizer) for i in tmp])
    tmp = tmp.view(1, -1).to(device)

    X.append(tmp)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [17]:
X = torch.stack(X)

In [18]:
y = []
for element in dataset:
    tmp = extract_features(element[1], model, tokenizer)
    y.append(tmp)
y = torch.stack(y, dim=0)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

Собственно, обучение:

In [20]:
for epoch in range(epochs):
    train_loss = 0.0
    vmmodel.train()
    optimizer.zero_grad()

    outputs = vmmodel(X_train)
    loss = criterion(outputs, y_train.to(device))
    train_loss += loss.item()
    loss.backward()
    optimizer.step()

    model.eval()
    valid_loss = 0.0
    target = vmmodel(X_test)
    loss = criterion(target, y_test.to(device))
    valid_loss += loss.item()

    print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_loss / len(X_train)}, Val Loss: {valid_loss / len(X_test)}")
        

Epoch [0/100], Train Loss: 0.005220823774212285, Val Loss: 0.09463133982249669
Epoch [1/100], Train Loss: 0.03591613706789518, Val Loss: 0.020926320127078464
Epoch [2/100], Train Loss: 0.007894709706306458, Val Loss: 0.01626767005239214
Epoch [3/100], Train Loss: 0.006013981997966766, Val Loss: 0.012080156377383642
Epoch [4/100], Train Loss: 0.004388577843967237, Val Loss: 0.009339992489133562
Epoch [5/100], Train Loss: 0.0034143532577313876, Val Loss: 0.007629721824611936
Epoch [6/100], Train Loss: 0.002811625991996966, Val Loss: 0.0060862089906420025
Epoch [7/100], Train Loss: 0.002196278619138818, Val Loss: 0.0053069985338619775
Epoch [8/100], Train Loss: 0.0018547236135131435, Val Loss: 0.004644189029932022
Epoch [9/100], Train Loss: 0.0015787726366206219, Val Loss: 0.003980914662991252
Epoch [10/100], Train Loss: 0.0013316116834941663, Val Loss: 0.0036535728722810745
Epoch [11/100], Train Loss: 0.0012180782658489126, Val Loss: 0.0034147330692836215
Epoch [12/100], Train Loss: 0.00

/home/user1/environments/hack/lib/python3.10/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([19, 1024])) that is different to the input size (torch.Size([19, 1, 1024])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/user1/environments/hack/lib/python3.10/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([7, 1024])) that is different to the input size (torch.Size([7, 1, 1024])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [21]:
len(X)

26

## Generating decoder

Так как из класса, описанного в предыдущем модуле мы получаем эмбеддинг предполагаемого ответа, необходимо расшифровать его. В таких случаях нам пригодится архитектура Encoder-Decoder. Encoder нам известен, осталось только написать Decoder

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

class EncoderBERT(nn.Module):
    def __init__(self, bert_model='bert-base-uncased', hidden_size=512):
        super(EncoderBERT, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model)
        self.fc = nn.Linear(self.bert.config.hidden_size, hidden_size)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # Берем [CLS] токен
        return self.fc(cls_embedding)  # Преобразуем размер для передачи в декодер

class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers=2):
        super(DecoderLSTM, self).__init__()
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc_out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_output, target_length):
        outputs = []
        decoder_input = encoder_output.unsqueeze(1)  # Начальный вход — скрытое состояние энкодера
        hidden_state = None
        
        for _ in range(target_length):
            output, hidden_state = self.lstm(decoder_input, hidden_state)
            prediction = self.fc_out(output.squeeze(1))
            outputs.append(prediction)
            decoder_input = output  # Используем предсказанный токен на следующем шаге
            
        return torch.stack(outputs, dim=1)

# Модель энкодера-декодера
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_ids, attention_mask, target_length):
        encoder_output = self.encoder(input_ids, attention_mask)
        output = self.decoder(encoder_output, target_length)
        return output

# Пример инициализации
encoder = EncoderBERT()
decoder = DecoderLSTM(hidden_size=512, output_size=30522)  # размер словаря BERT
model = Seq2Seq(encoder, decoder)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text = "Sample text for encoding."
inputs = tokenizer(text, return_tensors='pt')
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
target_length = 20  # Длина последовательности для генерации

# Генерация текста
with torch.no_grad():
    generated = model(input_ids, attention_mask, target_length=target_length)
    predictions = generated.argmax(dim=-1)
    decoded_text = tokenizer.decode(predictions[0], skip_special_tokens=True)
    print("Generated Text:", decoded_text)


Данный подход видится нам экспериментальным и не обязательно видится сто процентно верным, но перспективным за счет гибкости в обучении